# To recreate

First run the j17084truckduck_host script on the UTHP:

```bash
j17084truckduck_host > j1708overpru.log
```

Then grab a saleae logic pro 2, and capture the frames at the same time. Export the capture as a CSV file.


# Parse the data

In [8]:
import pandas as pd

csv_file_path = './j1708saleae-serial.csv'
data = pd.read_csv(csv_file_path)

filtered_data = data[data['type'] == 'data'].reset_index(drop=True)

# Initialize a list to hold frames
frames = []
current_frame = []

# Group frames by the MID byte (0x88, AC) ... which is a valid MID
for _, row in filtered_data.iterrows():
    if row['data'] in ('0x88','0xAC') and current_frame:
        frames.append(current_frame)
        current_frame = []
    current_frame.append(row['data'])
if current_frame:
    frames.append(current_frame)

# Convert each frame to a string and remove '0x' prefix
parsed_frames = ["".join([byte[2:] for byte in frame]) for frame in frames]

# Save the parsed frames to a log file
output_csv_parsed_path = 'parsed-j1708-over-saleae.log'
with open(output_csv_parsed_path, 'w') as file:
    file.write("\n".join(parsed_frames))


In [9]:
log_file_path = './j1708overpru.log'

# Extract lines with "Received from PRU:" and isolate the frame data
with open(log_file_path, 'r') as file:
    log_lines = file.readlines()

frames_pru = [line.split(": ")[1].strip() for line in log_lines if line.startswith("Received from PRU:")]
frames_pru_upper = [frame.upper() for frame in frames_pru]
output_log_parsed_path = 'parsed-j1708-over-pru.log'
with open(output_log_parsed_path, 'w') as file:
    file.write("\n".join(frames_pru_upper))


# Analysis

In [10]:
import difflib

# Read the contents of both log files
with open(output_csv_parsed_path, 'r') as file1, open(output_log_parsed_path, 'r') as file2:
    file1_contents = file1.readlines()
    file2_contents = file2.readlines()

# Compare the contents
differences = list(difflib.unified_diff(file1_contents, file2_contents, fromfile='saleae', tofile='pru'))

# Print the differences
for line in differences:
    print(line)

--- saleae

+++ pru

@@ -1,4 +1,3 @@

-FF

 AC0A004A

 88310D973FFF5400A80A01D101FF8D

 88310D973FFFA80A01D101FFE1

@@ -245,4 +244,4 @@

 88310D973FFFA80A01D101FFE1

 88310D973FFFA80A01D101FFE1

 88310D973FFFA80C01D101FFDF

-88310D973FFFA80A01D101FFE100
+88310D973FFFA80A01D101FFE1


... At this point the differences are due to the saleae logic pro 2 capturing wrong data. For example, FF MID is invalid, and the 00 byte was sampled at the exact point the j1708 bus was halted.

Therefore, the UTHP can capture j1708 messages correctly, and log them in real-time. 

Not, only can you log, J1708 or PLC4TRUCKS, but you can also send messages to the bus. Below is a comparison of the data captured by the UTHP and the Saleae Logic Pro 2.

# Parse the data

In [12]:
import pandas as pd

# Load the CSV file
csv_file_path = './j1708sent.csv'
data = pd.read_csv(csv_file_path)

# Filter rows with type 'data' and reset index
filtered_data = data[data['type'] == 'data'].reset_index(drop=True)

# Initialize a list to hold frames
frames = []
current_frame = []

# Group frames by the MID bytes (0x88 or 0x48) ... 48 was sent by the UTHP
for _, row in filtered_data.iterrows():
    if row['data'] in ('0x88', '0x48') and current_frame:
        frames.append(current_frame)
        current_frame = []
    current_frame.append(row['data'])
if current_frame:
    frames.append(current_frame)
    
# Convert each frame to a string and remove '0x' prefix
parsed_frames = ["".join([byte[2:] for byte in frame]) for frame in frames]

# Save the parsed frames to a log file
output_csv_parsed_path = 'parsed-j1708-sent-saleae.log'
with open(output_csv_parsed_path, 'w') as file:
    file.write("\n".join(parsed_frames))


In [13]:
log_file_path = './j1708sent.log'

# Extract lines with "Received from PRU:" and isolate the frame data
with open(log_file_path, 'r') as file:
    log_lines = file.readlines()

frames_pru = [line.split(": ")[1].strip() for line in log_lines if line.startswith("Received from PRU:")]
frames_pru_upper = [frame.upper() for frame in frames_pru]
output_log_parsed_path = 'parsed-j1708-sent-pru.log'
with open(output_log_parsed_path, 'w') as file:
    file.write("\n".join(frames_pru_upper))


In [14]:
def hex_to_ascii(hex_string):
    bytes_object = bytes.fromhex(hex_string)
    ascii_string = bytes_object.decode("ASCII", errors="ignore")
    return ascii_string

# Read lines from the log file
with open('./parsed-j1708-sent-pru.log', 'r') as log_file:
    log_lines = log_file.readlines()

# Convert each line from hex to ASCII
ascii_lines = [hex_to_ascii(line.strip()) for line in log_lines]

# Print the converted lines
for ascii_line in ascii_lines:
    print(ascii_line)

?
?

?
?


?
?	
?
?
Hello, PLC!
?	
?
?

?
?

?

?
?
?
5{

?
?
?


Cool, now we can see that the UTHP sent, "Hello, PLC!" to the bus.